In [ ]:
from RawHandler.RawHandler import RawHandler
from RawHandler.utils import download_file_requests
import matplotlib.pyplot as plt
import os

In [ ]:
# Download example raw files
def download_file_if_needed(file_url, output_file):
    if not os.path.exists(output_file):
        download_file_requests(file_url, output_file)

file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/UNYG7K"
output_file = "gtBark_GT.arw"
download_file_if_needed(file_url, output_file)

In [ ]:
# Load the first image
rh = RawHandler("gtBark_GT.arw")

In [ ]:
# Shows the raw pattern
rh.bayer_pattern_description()

In [ ]:
# We can acccess the underlying bayer data
# dims = (h1, h2, w1, w2)
dims = (1500, 1500 + 200, 4500, 4500 + 200)
bayer = rh.raw
plt.imshow(bayer[dims[0] : dims[1], dims[2] : dims[3]])

In [ ]:
# Or, we can use the input_handler method to crop it and format it as shape [1, H, W] as needed for using in common machine learning libraries.
# The cropping functionality rounds to n % 2 == 0 to ensure proper alighment of the bayer array.
bayer = rh.input_handler(dims=dims)
plt.imshow(bayer.transpose(1, 2, 0))
bayer.min(), bayer.max()

In [ ]:
# To return a representation that is black and white point corrected, as well as scaled (roughly) between 0 and 1, we can use the 'adjust_bayer_bw_levels' method.
bayer = rh.adjust_bayer_bw_levels(dims=dims)
plt.imshow(bayer.transpose(1, 2, 0))
bayer.min(), bayer.max()

In [ ]:
# However, chances are we want two representations: the rggb resentation of the bayer array and a demosaiced ground truth.
# The rggb representation stacks the 4 pixels of the bayer pattern into one. Naturally, this has 1/2 the linear size of the original image.
rggb = rh.as_rggb(dims=dims)
plt.subplots(1, 4, figsize=(16, 4))
for idx, channel in enumerate(rh.bayer_pattern):
    plt.subplot(1, 4, idx + 1)
    plt.imshow(rggb[idx])
    plt.title(channel)

In [ ]:
# We might also want a demosaiced version as an input or ground truth.
# Since the output is still in a linear space with no gamma curve applied, I multiply by 3 for visualization.
rgb = rh.as_rgb(dims=dims)
plt.imshow(rgb.transpose(1, 2, 0) * 3)

In [ ]:
# Of course, that looks quite green because we haven't converted the camera colorspace to one we use for visualization, but we can do that as well!
aRGB = rh.as_RGB_colorspace(dims=dims, colorspace="AdobeRGB")
plt.imshow(aRGB.transpose(1, 2, 0) * 3)

In [ ]:
# Each call process the image from the basic raw data, if we do not want to do this, we can specify the input image with the 'img' keyword arg.
aRGB = rh.as_RGB_colorspace(img=rgb, colorspace="AdobeRGB")
plt.imshow(aRGB.transpose(1, 2, 0) * 3)